# Transform and Load preparion steps:
### * 1) Run create_db.sql script
### * 2) Export keys_tables:
    * 2A) Run create_keys_table_1.sql script
    * 2B) Export tables as csv and store on local drive
### * 3) Read in all keys_tables and create dfs
### * 4) Next follow procedures below to load data into database


## NOTE: ORDER OF LOADING DATABASE MATTERS

### 1. Game table should be loaded first

### 2. Players table should be loaded second

### 3. After game table and players table loaded to database, export game_2017 and players_2017 as csv and read in. 
### * These tables contain primary key columns needed for loading player stats and team stats. These are two additional keys_tables

### 4. player_stats/team_stats should be loaded last 

In [95]:
import pandas as pd
import os
from mysql_conn import password
from sqlalchemy import create_engine
import datetime

In [96]:
# Import keys csvs

positions_df = pd.read_csv("keys_table_2017/positions-list-2017.csv", encoding="UTF-8")
team_df = pd.read_csv("keys_table_2017/team-list-2017.csv", encoding="UTF-8")
region_df = pd.read_csv("keys_table_2017/region-list-2017.csv", encoding="UTF-8")
tournament_round_df = pd.read_csv(
    "keys_table_2017/tournament_round-list-2017.csv", encoding="UTF-8")

In [97]:
# scorebox, winner_team_stats, loser_team_stats csvs

scorebox_csv = "Resources/2017_03_14_kansas_st_wake_forest_boxscore_game_results.csv"
scorebox_df = pd.read_csv(scorebox_csv, encoding="UTF=8")

winner_csv = "Resources/2017_03_14_kansas_st_wake_forest_boxscore_Kansas St.csv"
winner_df = pd.read_csv(winner_csv, encoding="UTF-8")

loser_csv = "Resources/2017_03_14_kansas_st_wake_forest_boxscore_Wake Forest.csv"
loser_df = pd.read_csv(loser_csv, encoding="UTF-8")

In [98]:
# pull in data from data world

import datadotworld as dw

# Load the data object
data_object = dw.load_dataset('rustygentile/ncaa-etl', force_update=True)
available_csvs = list(data_object.dataframes)

# print out the first five csvs:
available_csvs[:5]

# load 3 dataframes as an example
dfs_2017 = []
for csvs in available_csvs[:3]:
    if "2017" in csvs:
        dfs_2017.append(data_object.dataframes[csvs])

for d in dfs_2017:
    print("Dataframe:")
    print(d.head())

Dataframe:
   column_a  column_b    column_c
0         1         2  Unnamed: 3
1        40        55          95
2        36        52          88
Dataframe:
             name pos min fgm_a 3pm_a  ftm_a  oreb  reb  ast   st  blk   to  \
0   Wesley Iwundu   F  35   6-9   1-2  11-13   1.0  6.0  7.0  1.0  0.0  2.0   
1    D.J. Johnson   F  34   8-9   0-0    2-2   3.0  6.0  0.0  0.0  3.0  0.0   
2    Xavier Sneed   F  12   0-2   0-2    2-4   0.0  2.0  2.0  4.0  0.0  2.0   
3  Isaiah Maurice   F   6   0-0   0-0    0-0   0.0  1.0  0.0  0.0  0.0  1.0   
4       Dean Wade   F  23   3-4   0-1    2-3   0.0  2.0  2.0  0.0  0.0  2.0   

    pf   pts  
0  3.0  24.0  
1  4.0  18.0  
2  3.0   2.0  
3  3.0   0.0  
4  2.0   8.0  
Dataframe:
                    name pos min fgm_a 3pm_a ftm_a  oreb  reb  ast   st  blk  \
0     Sam Japhet-Mathias   C   2   0-0   0-0   0-0   0.0  0.0  0.0  0.0  0.0   
1  Konstantinos Mitoglou   F  29   1-5   1-4   4-6   1.0  2.0  0.0  1.0  0.0   
2         Greg McClinton  

## Loading Step 1: Transform and load game table

### keys_tables needed:

* tournament_round_2017 - keys_table
* team_2017 - keys_table

* Dynamically pull in all 2017 scorebox_csv files

* Loop thru files, then transform and load to database

In [99]:
"""--------------------------------------------------------------"""
# Get date
date = scorebox_csv.split('/')
date = date[-1]
date = date.split('_boxscore_game_results.csv')[0]
date = date.split('_')
year = date[0]
month = date[1]
day = date[2]
date = f"{year}-{month}-{day}"

# Get winner team
winner = winner_csv.split('_')
winner = winner[-1]
winner_team = str(winner.split('.csv')[0])
winner_team

# Get loser team
loser = loser_csv.split('_')
loser = loser[-1]
loser_team = str(loser.split('.csv')[0])
loser_team

# Get game
team_play = f"{winner_team} vs. {loser_team}"
team_play

# rename scorebox columns
cols = ['1', '2', 'Unnamed: 3']
scorebox = pd.DataFrame(scorebox_df, columns=(cols))
score_df = scorebox.rename(
    columns={'1': 'date', '2': 'teams', 'Unnamed: 3': 'score'}).copy()
score_df['teams'][0] = winner_team
score_df['teams'][1] = loser_team

# transform date column to datetime and insert into table
score_df['date'] = pd.to_datetime(score_df['date'])
score_df['date'][0] = date
score_df['date'][1] = date

# transform tournament round date column to datetime type
tournament_round_df['date'] = pd.to_datetime(tournament_round_df['date'])

# create copy of teams_df and rename columns
team_df_alter = team_df
team_df_alter = team_df_alter.rename(columns={'name': 'teams'})

# create game dataframe by merging score_df and team_df
game = pd.merge(score_df, team_df_alter, on="teams")

# make game region_id column double type
game['region_id'] = pd.to_numeric(game['region_id'], downcast='float')

# merge game table with tournament table
game_df = pd.merge(score_df, tournament_round_df, on="date")
game_df = game_df.rename(columns={'id': 'tournament_round_id'})
game_df['score'] = game_df['score'].drop_duplicates()
game_df = game_df.dropna(how='any')

# convert region_id and score to int type
game_df['region_id'] = pd.to_numeric(game_df['region_id'], downcast='integer')
game_df['score'] = pd.to_numeric(game_df['score'], downcast='integer')

# create variables for new game_df containing a single row of data
# tournament_round_id
tournament_round_id = game_df['tournament_round_id'][0]
# region_id
region_id = game_df['region_id'][0]
# team_play
team_play = team_play
# winner
winner = winner_team
# loser
loser = loser_team
# score
score_winner = score_df['score'][0]
score_loser = score_df['score'][1]
score = f"{score_winner}-{score_loser}"

# create new game dataframe
cols = ["tournament_round_id", "region_id",
        "team_play", "winner", "loser", "score"]
game_table = pd.DataFrame(columns=cols)

game_table = game_table.append({"tournament_round_id": tournament_round_id,
                                "region_id": region_id,
                                "team_play": team_play,
                                "winner": winner,
                                "loser": loser,
                                "score": score}, ignore_index=True)

# Load to database
rds_connection_string = f"root:{password}@127.0.0.1/ncaam_basketball_2017_db"
engine = create_engine(f'mysql://{rds_connection_string}')
# CONFIRM TABLES
engine.table_names()

game_table.to_sql(
    name='game', con=engine,
    if_exists='append', index=False)

C:\Users\axand\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\axand\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\axand\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [6]:
# transform and load players tables

# populate players table second

# csvs needed:

# 1) all 2017 and specific dates that all happened on first four or round one games



In [100]:
# Transform and load winner players table

# create player_loser_table:
player_winner_table = pd.DataFrame(winner_df[['Name', 'POS']][:-2])

# Rename position_df name column
positions_df = positions_df.rename(columns={'abrv': 'POS'})

# MERGE player with position on on 'POS'

player_winner_table_1 = pd.merge(player_winner_table, positions_df, on="POS")

player_winner_table_1 = player_winner_table_1.rename(
    columns={'id': 'position_id'})

# Create team column forlwinner team

player_winner_table_1['team'] = winner_team

# transform teams column name
team_alter_2 = team_df

team_alter_2 = team_alter_2.rename(columns={'name': 'team'})

# merge team table

player_winner_table_combined = pd.merge(
    player_winner_table_1, team_alter_2, on='team')

player_winner_table_combined = player_winner_table_combined.rename(
    columns={'name': 'position_name', 'id': 'team_id', 'Name': 'name'})

player_winner_table_combined

# transfor player table to export to database

player_winner_table_transformed = player_winner_table_combined[[
    'name', 'position_id', 'team_id']]

player_winner_table_transformed

# CREATE DATABASE connection

rds_connection_string = f"root:{password}@127.0.0.1/ncaam_basketball_2017_db"
engine = create_engine(f'mysql://{rds_connection_string}')

# CONFIRM TABLES

engine.table_names()

# load data to database

player_winner_table_transformed.to_sql(
    name='players', con=engine,
    if_exists='append', index=False)

In [101]:
# Rinse and Repeat for loser players table

# create player_loser_table:
player_loser_table = pd.DataFrame(loser_df[['Name', 'POS']][:-2])

# MERGE player with position on POS

player_loser_table_1 = pd.merge(player_loser_table, positions_df, on="POS")

player_loser_table_1 = player_loser_table_1.rename(
    columns={'id': 'position_id'})

# Create team column for loser team

player_loser_table_1['team'] = loser_team

# transform teams column name
team_alter_3 = team_df

team_alter_3 = team_alter_3.rename(columns={'name': 'team'})

# merge team table

player_loser_table_combined = pd.merge(
    player_loser_table_1, team_alter_3, on='team')

player_loser_table_combined = player_loser_table_combined.rename(
    columns={'name': 'position_name', 'id': 'team_id', 'Name': 'name'})

player_loser_table_combined

# transfor player table to export to database

player_loser_table_transformed = player_loser_table_combined[[
    'name', 'position_id', 'team_id']]

player_loser_table_transformed

# CREATE DATABASE connection

rds_connection_string = f"root:{password}@127.0.0.1/ncaam_basketball_2017_db"
engine = create_engine(f'mysql://{rds_connection_string}')

# CONFIRM TABLES

engine.table_names()

# load data to database

player_loser_table_transformed.to_sql(
    name='players', con=engine,
    if_exists='append', index=False)

# ATTENTION: 
## * Users will need the player_key_table and team_key_table:
     * Run create_keys_table_2.sql script 
     * Export tables as csv and store on local drive
## * Once csvs are saved to local drive then proceed to next steps

## Additional keys_tables:
* player-list-2017 - keys_table
* game-list-2017 - keys_table

In [102]:
# read in player_table and game_table
game_df = pd.read_csv("keys_table_2017/game-list-2017.csv", encoding="UTF-8")
player_df = pd.read_csv("keys_table_2017/player-list-2017.csv", encoding="UTF-8")

### Transform and Load winner players_stats table

In [103]:
# create players_stats_winner datatframe

players_stats_winner = dfs_2017[1][:-2]

# split stat cols

new_fg = players_stats_winner["fgm_a"].str.split("-", n=1, expand=True)
new_3p = players_stats_winner["3pm_a"].str.split("-", n=1, expand=True)
new_ft = players_stats_winner["ftm_a"].str.split("-", n=1, expand=True)

players_stats_winner['fgm'] = new_fg[0]
players_stats_winner['fga'] = new_fg[1]

players_stats_winner['3pm'] = new_3p[0]
players_stats_winner['3pa'] = new_3p[1]


players_stats_winner['ftm'] = new_ft[0]
players_stats_winner['fta'] = new_ft[1]

players_stats_winner_df = players_stats_winner


# Merge player table
players_stats_winner_df = pd.merge(players_stats_winner, player_df, on='name')

# add game_id

game_id = game_df['id'][0]
players_stats_winner_df['game_id'] = game_id
players_stats_winner_df = players_stats_winner_df.rename(
    columns={'id': 'player_id'})

# transform players_stats_winner database

players_stats_winner_transformed = players_stats_winner_df[['player_id', 'game_id', 'min', 'fgm', 'fga', '3pa', '3pm',
                                                            'fta', 'ftm', 'oreb', 'reb', 'ast', 'st', 'blk', 'to', 'pf', 'pts']].astype(int)

players_stats_winner_transformed = players_stats_winner_transformed.rename(columns={'min': 'Min', 'fgm': 'FGM',
                                                                                    'fga': 'FGA', '3pa': '3PA', '3pm': '3PM',
                                                                                    'fta': 'FTA', 'ftm': 'FTM',
                                                                                    'oreb': 'OREB', 'reb': 'REB',
                                                                                    'ast': 'AST', 'st': 'ST', 'to': 'Turnover',
                                                                                    'pf': 'PF', 'pts': 'PTS', 'blk': 'BLK'})

# CREATE DATABASE connection

rds_connection_string = f"root:{password}@127.0.0.1/ncaam_basketball_2017_db"
engine = create_engine(f'mysql://{rds_connection_string}')

# load data to database

players_stats_winner_transformed.to_sql(
    name='player_stats', con=engine,
    if_exists='append', index=False)

C:\Users\axand\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\axand\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
C:\Users\axand\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co

### Transform and Load loser players_stats table

In [104]:
# RINSE AND REPEAT FOR loser PLAYERS_STATS table

# create players_stats_loser datatframe

players_stats_loser = dfs_2017[2][:-2]

# split stat cols

new_fg = players_stats_loser["fgm_a"].str.split("-", n=1, expand=True)
new_3p = players_stats_loser["3pm_a"].str.split("-", n=1, expand=True)
new_ft = players_stats_loser["ftm_a"].str.split("-", n=1, expand=True)

players_stats_loser['fgm'] = new_fg[0]
players_stats_loser['fga'] = new_fg[1]

players_stats_loser['3pm'] = new_3p[0]
players_stats_loser['3pa'] = new_3p[1]


players_stats_loser['ftm'] = new_ft[0]
players_stats_loser['fta'] = new_ft[1]

players_stats_loser_df = players_stats_loser


# Merge player table

players_stats_loser_df = pd.merge(players_stats_loser, player_df, on='name')

# add game_id

game_id = game_df['id'][0]
players_stats_loser_df['game_id'] = game_id
players_stats_loser_df = players_stats_loser_df.rename(
    columns={'id': 'player_id'})

# transform players_stats_loser for database

players_stats_loser_transformed = players_stats_loser_df[['player_id', 'game_id', 'min', 'fgm', 'fga', '3pa', '3pm',
                                                          'fta', 'ftm', 'oreb', 'reb', 'ast', 'st', 'blk', 'to', 'pf', 'pts']].astype(int)

players_stats_loser_transformed = players_stats_loser_transformed.rename(columns={'min': 'Min', 'fgm': 'FGM',
                                                                                  'fga': 'FGA', '3pa': '3PA', '3pm': '3PM',
                                                                                  'fta': 'FTA', 'ftm': 'FTM',
                                                                                  'oreb': 'OREB', 'reb': 'REB',
                                                                                  'ast': 'AST', 'st': 'ST', 'to': 'Turnover',
                                                                                  'pf': 'PF', 'pts': 'PTS', 'blk': 'BLK'})

# CREATE DATABASE connection

rds_connection_string = f"root:{password}@127.0.0.1/ncaam_basketball_2017_db"
engine = create_engine(f'mysql://{rds_connection_string}')

# load data to database

players_stats_loser_transformed.to_sql(
    name='player_stats', con=engine,
    if_exists='append', index=False)

C:\Users\axand\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
C:\Users\axand\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\axand\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

### Transform and Load winner team_stats table

In [107]:
# CREATE NEW DATAFRAME
team_stats_winner_df = players_stats_winner_transformed

# create summary varibles

fgm_sum = team_stats_winner_df['FGM'].sum()
fga_sum = team_stats_winner_df['FGA'].sum()
three_pa = team_stats_winner_df['3PA'].sum()
three_pm = team_stats_winner_df['3PM'].sum()
ftm_sum = team_stats_winner_df['FTM'].sum()
fta_sum = team_stats_winner_df['FTA'].sum()
oreb_sum = team_stats_winner_df['OREB'].sum()
reb_sum = team_stats_winner_df['REB'].sum()
ast_sum = team_stats_winner_df['AST'].sum()
st_sum = team_stats_winner_df['ST'].sum()
blk_sum = team_stats_winner_df['BLK'].sum()
turnover_sum = team_stats_winner_df['Turnover'].sum()
pf_sum = team_stats_winner_df['PF'].sum()
pts_sum = team_stats_winner_df['PTS'].sum()


# CREATE DATA FRAME
team_stats_winner_summary = pd.DataFrame({'FGM': [fgm_sum],
                                          'FGA': [fga_sum],
                                          '3PA': [three_pa],
                                          '3PM': [three_pm],
                                          'FTM': [ftm_sum],
                                          'FTA': [fta_sum],
                                          'OREB': [oreb_sum],
                                          'REB': [reb_sum],
                                          'AST': [ast_sum],
                                          'ST':  [st_sum],
                                          'BLK': [blk_sum],
                                          'Turnover': [turnover_sum],
                                          'PF': [pf_sum],
                                          'PTS': [pts_sum]
                                          })

# add game_id and team_id
team_stats_winner_summary['game_id'] = game_id = game_df['id'][0]
team_stats_winner_summary['team_id'] = player_winner_table_transformed['team_id'][0]

# CREATE DATABASE connection

rds_connection_string = f"root:{password}@127.0.0.1/ncaam_basketball_2017_db"
engine = create_engine(f'mysql://{rds_connection_string}')

# load data to database
team_stats_winner_summary.to_sql(
    name='team_stats', con=engine,
    if_exists='append', index=False)


team_stats_winner_summary

,FGM,FGA,3PA,3PM,FTM,FTA,OREB,REB,AST,ST,BLK,Turnover,PF,PTS,game_id,team_id
0,31,47,16,6,27,36,6,28,18,6,3,15,26,95,1,4


### Transform and Load loser team_stats table

In [108]:
# CREATE NEW DATAFRAME
team_stats_loser_df = players_stats_loser_transformed

# create summary varibles

fgm_sum = team_stats_loser_df['FGM'].sum()
fga_sum = team_stats_loser_df['FGA'].sum()
three_pa = team_stats_loser_df['3PA'].sum()
three_pm = team_stats_loser_df['3PM'].sum()
ftm_sum = team_stats_loser_df['FTM'].sum()
fta_sum = team_stats_loser_df['FTA'].sum()
oreb_sum = team_stats_loser_df['OREB'].sum()
reb_sum = team_stats_loser_df['REB'].sum()
ast_sum = team_stats_loser_df['AST'].sum()
st_sum = team_stats_loser_df['ST'].sum()
blk_sum = team_stats_loser_df['BLK'].sum()
turnover_sum = team_stats_loser_df['Turnover'].sum()
pf_sum = team_stats_loser_df['PF'].sum()
pts_sum = team_stats_loser_df['PTS'].sum()


# CREATE DATA FRAME
team_stats_loser_summary = pd.DataFrame({'FGM': [fgm_sum],
                                         'FGA': [fga_sum],
                                         '3PA': [three_pa],
                                         '3PM': [three_pm],
                                         'FTM': [ftm_sum],
                                         'FTA': [fta_sum],
                                         'OREB': [oreb_sum],
                                         'REB': [reb_sum],
                                         'AST': [ast_sum],
                                         'ST':  [st_sum],
                                         'BLK': [blk_sum],
                                         'Turnover': [turnover_sum],
                                         'PF': [pf_sum],
                                         'PTS': [pts_sum]
                                         })

# add game_id and team_id
team_stats_loser_summary['game_id'] = game_id = game_df['id'][0]
team_stats_loser_summary['team_id'] = player_loser_table_transformed['team_id'][0]


# CREATE DATABASE connection

rds_connection_string = f"root:{password}@127.0.0.1/ncaam_basketball_2017_db"
engine = create_engine(f'mysql://{rds_connection_string}')

# load data to database
team_stats_loser_summary.to_sql(
    name='team_stats', con=engine,
    if_exists='append', index=False)


team_stats_loser_summary

,FGM,FGA,3PA,3PM,FTM,FTA,OREB,REB,AST,ST,BLK,Turnover,PF,PTS,game_id,team_id
0,25,53,27,9,29,32,6,19,14,8,1,11,25,88,1,1
